In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pickle

In [2]:
force_recalculation = True

In [3]:
# Loading data
recensement = pd.read_excel(io="../../1832_v4.xlsx", na_values=["·","?"])
classes = pd.read_excel(io="../../histoire urbaine digitale classification metiers.xlsx", sheet_name="data",  na_values="·")

# Adding the 'chef_vocation_categorie' column
classes = classes.rename(columns={"JOB": "chef_vocation_norm_2"})
classes = classes.rename(columns={"CLASS": "chef_vocation_categorie"})
recensement = recensement.merge(classes, on="chef_vocation_norm_2", how="outer")

# Split the values on the '/' character
split_vals = recensement['chef_vocation_categorie'].str.split('/')

# Create a new dataframe from the split values
recensement = recensement.assign(chef_vocation_categorie=split_vals).explode('chef_vocation_categorie')

Only a subset is taken to make the graph. Comment this line to study the full graph.

In [4]:
def filter_most_common(df: pd.DataFrame, row_name: str, n: int) -> pd.DataFrame:
    value_counts = df[row_name].value_counts()
    most_common = value_counts.index[:n]
    return df[df[row_name].isin(most_common)]


Filtering some data to reduce complexity of the network.

In [5]:
original_size = len(recensement)

# Keep only the persons who have a classified vocation
recensement = recensement.query("not chef_vocation_categorie.isna()")
print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

# Keep only the persons whose rue is among the 10 most common
# recensement = filter_most_common(recensement, "nom_rue_norm_2", 1000)
# print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

# Keep only the persons whose origine is among the 10 most common
# recensement = filter_most_common(recensement, "chef_origine_norm_2", 1000)
# print(f"Size is now {len(recensement)/original_size*100:.0f}% of the original.")

Size is now 80% of the original.


A `networkx` graph is made. If the file `individus_origine_rue.pkl` already exists, the graph is loaded from there. Else it is recalculated (can take up to 5 minutes if the full graph is being used).

In [6]:
if force_recalculation or not os.path.exists("individus_origine_rue.pkl"):
    G = nx.Graph()
    for index, row in recensement.iterrows():
        G.add_node(
            index,
            nom_rue=row["nom_rue_norm_2"],
            proprietaire=row["proprietaire_nom_norm_2"],
            chef_nom=row["chef_nom_norm_2"],
            chef_annee_naissance=row["chef_annee_naissance"],
            chef_origine=row["chef_origine_norm_2"],
            chef_vocation=row["chef_vocation_norm_2"],
            chef_vocation_categorie=row["chef_vocation_categorie"],
        )
    for index1, row1 in tqdm(recensement.iterrows(), total=len(recensement)):
        if pd.isna(row1["chef_vocation_categorie"]): continue
        for index2, row2 in recensement.iterrows():
            if pd.isna(row2["chef_vocation_categorie"]): continue
            if index2 < index1: continue

            score = 0

            #if row1["chef_vocation_categorie"] == row2["chef_vocation_categorie"]: score += 1
            # if row1["chef_vocation_norm_2"] == row2["chef_vocation_norm_2"]: score += 1
            # if row1["proprietaire_nom_norm_2"] == row2["proprietaire_nom_norm_2"]: score += 1
            if row1["nom_rue_norm_2"] == row2["nom_rue_norm_2"]: score += 1
            if row1["chef_origine_norm_2"] == row2["chef_origine_norm_2"]: score += 1
            #if row1["nom_rue_norm_2"] == row2["nom_rue_norm_2"]: score += 1

            if score > 0: G.add_edge(index1, index2, weight=score)
            
    pickle.dump(obj=G, file=open("individus_origine_rue.pkl", "wb"))
else:
    G = pickle.load(file=open("individus_origine_rue.pkl", "rb"))

100%|██████████| 3071/3071 [05:13<00:00,  9.81it/s]


Write `.gexf` file.

In [7]:
nx.write_gexf(G, 'individus_origine_rue.gexf')